## Baby Name Generaton
<p>A n-order Markov model to generate novel names that follow common letter sequences.</p>

In [5]:
import numpy as np
np.random.seed(0)

### Read 1k boys and girls names, respectively. Build transition probabilities using dictionary

In [6]:
boys = np.loadtxt('namesBoys.txt',dtype='|S10')
girls = np.loadtxt('namesGirls.txt',dtype='|S10')

In [97]:
# build nested dictionaries that record number of letter occurences for previous n letters
# E.G.{previous three letters: {letter1:num occurance,letter2:num occurance}}
# E.G. {'str':{'a':3,'c':5,...,'z':1,'_':10}}
def get_occurrence(names_list, order):
    dic = {} 
    # loop through each name
    for name in names_list:
        # loop through each letter
        for i in range(len(name)):
            # avoid index out of range
            if i >= order and i<= len(name)-1: 
                if name[i-order:i] not in dic:
                    # initiate dictionary for the <previous three letters>
                    dic[name[i-order:i]]={name[i]:1}
                else:
                    if name[i] not in dic[name[i-order:i]]:
                        dic[name[i-order:i]][name[i]] = 1
                    else:
                        # increment occurence of trailing letter by 1
                        dic[name[i-order:i]][name[i]] += 1 
    return dic

# build dictionary that contains trailing letter and corresponding transition probability for previous n letters
# E.G. {previous three letters: [[letter1,letter2,letter3],[0.25, 0.55, 0.3]]}
# E.G. {'str':[['a','c',...,'z','_'],[0.12,0.23,0.03,..]]}
def get_transition_prob(occurence_dic):
    transition_dic = {}
    # loop through each <previous three letters>
    for first_n, occurences in occurence_dic.iteritems():
        # initiate an empty list
        transition_dic[first_n] = []
        # get total number of occurences for all letters after these previous n letters
        total_occurences = sum(occurences.values())
        # get transition probabilities for all letters after these previous n letters
        transition_prob = map(lambda x:float(x)/total_occurences, occurences.values())
        transition_dic[first_n].append(occurences.keys())
        transition_dic[first_n].append(transition_prob)
    
    return transition_dic

# get next letter based on previous n letters and transition probabilities
def get_next_letter(first_n, trans_dic):
    return np.random.choice(trans_dic[first_n][0], 1, p=trans_dic[first_n][1])[0]

# get names given order of Markov model
def get_n_order_names(name_list, order, num_names, min_length, max_length):
    if order < 1:
        return 'Order should be larger than 1'
    elif order >= min_length:
        return 'Order should be smaller than min_length'
    elif min_length > max_length:
        return 'min_length cannot be larger than max_length'
    else:
        #name_list = filter(lambda x: len(x)>= order+1, name_list)
        # add "_" before and after the names, will be used to indicate the start and end of the names
        # if order is n, then there should be '_' * n before and after the name
        name_set = set(map(lambda x:'_'*order + x.lower() + '_'*order, name_list))
        
        # get num of occurrences of each single letter after the previous n letters
        occurrence_dic = get_occurrence(name_set, order)
    
        # get transition probability for each single lette afte the previous n letters
        trans_dic = get_transition_prob(occurrence_dic)
        
        # start generating names
        name_list = []
        # keep generating names until we had enough
        while len(name_list) < num_names:
            new_name = '_' * order

            # start from the first letter
            first_letter = get_next_letter('_' * order, trans_dic)
            new_name += first_letter

            # keep generating letters until reaching the end
            while new_name[-order:] != '_' * order:
                next_letter = get_next_letter(new_name[-order:], trans_dic)
                new_name += next_letter

            # only add new names that fulfill the requirements
            if new_name not in name_set and new_name not in name_list and \
            len(new_name) >= min_length+order*2 and len(new_name) <= max_length+order*2:
                name_list.append(new_name)

        # remove "___" in the names
        name_list = map(lambda x:x.replace('_','').title(),name_list)
    
    return name_list

In [148]:
order = 3
num_names = 5
min_length = 6 # at least 1, cannot be larger than max_length
max_length = 6 # at least 2
name_type = 'boy'

if name_type == 'boy':
    new_names = get_n_order_names(boys, order, num_names, min_length, max_length)
else:
    new_names = get_n_order_names(girls, order, num_names, min_length, max_length)

print new_names

['Jerman', 'Gordan', 'Deacob', 'Nigelo', 'Tayles']


### Serialize the transition probabilities to binary files, for future use

In [149]:
import pickle

#### write

In [571]:
with open('trans_boy.pickle', 'wb') as f:
    # serialize the dictionary using the highest protocol available, "wb" refers to "write binary"
    pickle.dump(trans_boy, f, pickle.HIGHEST_PROTOCOL)
    
with open('trans_girl.pickle', 'wb') as f:
    pickle.dump(trans_girl, f, pickle.HIGHEST_PROTOCOL)
    
with open('set_boys.pickle', 'wb') as f:
    pickle.dump(set_boys, f, pickle.HIGHEST_PROTOCOL)
    
with open('set_girls.pickle', 'wb') as f:
    pickle.dump(set_girls, f, pickle.HIGHEST_PROTOCOL)

#### read

In [572]:
with open('trans_boy.pickle', 'rb') as f:
    # Read pickle file back, "rb" refers to "read binary"
    trans_boy = pickle.load(f)
    
with open('trans_girl.pickle', 'rb') as f:
    trans_girl = pickle.load(f)
    
with open('set_boys.pickle', 'rb') as f:
    set_boys = pickle.load(f)
    
with open('set_girls.pickle', 'rb') as f:
    set_girls = pickle.load(f)